# Extract data from invitrodb 

In [64]:
from sqlalchemy import create_engine, inspect, text
import polars as pl
import gc

In [ ]:
# 'intended target family' would be a cool way to subset the cell-free assay types
# cell-free assays don't have paired viability assays (doesn't make sense for cellfree)
# use "cell_viability_assay", not "burst_assay" because burst_assay misses some
# Level 4 has the min / max concentration, plus number of concentrations tested

# 1:1 relationship between components and endpoints
# DECISION: treat bidirectional assays the same, with the motivation from MorphMap that knock-down/overexpression phenotypes are positively correlated

## Get assay and endpoint information

In [66]:
# Get assays of interest
engine = create_engine("mysql+pymysql://root@localhost/prod_internal_invitrodb_v4_1")

with engine.connect() as connection:
    result = connection.execute(text("SELECT * FROM assay"))
    column_names = result.keys()
    rows = result.fetchall()
    rows = [list(row) for row in rows]
    assay = pl.DataFrame(rows, schema=column_names)

print(assay.shape)

assay = assay.filter(pl.col("cell_format") != "whole embryo")
assay = assay.filter(pl.col("organism") == "human")
cell_assay = assay.filter(pl.col("cell_format").is_in(["cell-based", "cell line", "primary cell", "secondary cell", "primary cell co-culture"]))
cellfree_assay = assay.filter(pl.col("cell_format").is_in(["tissue-based cell-free", "cell-free"]))

# remove follow-up experiments
cellfree_assay = cellfree_assay.filter(~pl.col("assay_name").str.contains("Followup"))
cell_assay = cell_assay.filter(~pl.col("assay_name").str.contains("Followup"))
cell_assay = cell_assay.filter(~pl.col("assay_name").str.contains("TRANS"))
cell_assay = cell_assay.filter(~pl.col("assay_name").str.contains("EcoTox"))

cell_assay = cell_assay.filter(~pl.col("assay_name").is_in([
    "CLD_6hr", "CLD_24hr", "APR_HepG2_1hr", "APR_HepG2_72hr"]
    ))

# get aid lists
cell_aid = cell_assay.select("aid").to_series().unique().to_list()
cellfree_aid = cellfree_assay.select("aid").to_series().unique().to_list()

assay_keep = ["aid", "timepoint_hr", "cell_format", "tissue", "cell_short_name"]
cell_assay = cell_assay.select(assay_keep)
cellfree_assay = cellfree_assay.select(assay_keep)

(623, 18)


In [67]:
# Get assay components of interest
engine = create_engine("mysql+pymysql://root@localhost/prod_internal_invitrodb_v4_1")

with engine.connect() as connection:
    result = connection.execute(text("SELECT * FROM assay_component"))
    column_names = result.keys()
    rows = result.fetchall()
    rows = [list(row) for row in rows]
    component = pl.DataFrame(rows, schema=column_names).drop([
        "detection_technology_type", "detection_technology_type_sub", "detection_technology", 
        "key_assay_reagent_type", "key_assay_reagent", "technological_target_type", "technological_target_type_sub",
        "parameter_readout_type", "assay_component_target_desc", "assay_design_type_sub", "biological_process_target"
    ])

component = component.filter(~pl.col("assay_component_name").str.contains("_ch1"))
component = component.filter(~pl.col("assay_component_name").str.contains("_ch2"))
component = component.filter(~pl.col("assay_component_name").is_in([
    "TOX21_RT_HEPG2_FLO_00hr_viability",
    "TOX21_RT_HEPG2_FLO_08hr_viability",
    "TOX21_RT_HEPG2_FLO_16hr_viability",
    "TOX21_RT_HEPG2_FLO_24hr_viability",
    "TOX21_RT_HEPG2_FLO_32hr_viability",
    "TOX21_RT_HEPG2_GLO_00hr_viability",
    "TOX21_RT_HEPG2_GLO_08hr_viability",
    "TOX21_RT_HEPG2_GLO_16hr_viability",
    "TOX21_RT_HEPG2_GLO_24hr_viability",
    "TOX21_RT_HEPG2_GLO_32hr_viability",
    "TOX21_RT_HEK293_FLO_00hr_viability",
    "TOX21_RT_HEK293_FLO_08hr_viability",
    "TOX21_RT_HEK293_FLO_16hr_viability",
    "TOX21_RT_HEK293_FLO_24hr_viability",
    "TOX21_RT_HEK293_FLO_32hr_viability",
    "TOX21_RT_HEK293_GLO_00hr_viability",
    "TOX21_RT_HEK293_GLO_08hr_viability",
    "TOX21_RT_HEK293_GLO_16hr_viability",
    "TOX21_RT_HEK293_GLO_24hr_viability",
    "TOX21_RT_HEK293_GLO_32hr_viability",
]))
component = component.drop("assay_component_name")


cell_component = component.filter(pl.col("aid").is_in(cell_aid))
cell_component = cell_component.filter(pl.col("assay_design_type") != "background reporter")

# there is only one component per assay for the cell-free assays
cellfree_component = component.filter(pl.col("aid").is_in(cellfree_aid))

cell_acid = cell_component.select("acid").to_series().unique().to_list()
cellfree_acid = cellfree_component.select("acid").to_series().unique().to_list()


In [68]:
# Get assay endpoints of interest
engine = create_engine("mysql+pymysql://root@localhost/prod_internal_invitrodb_v4_1")

with engine.connect() as connection:
    result = connection.execute(text("SELECT * FROM assay_component_endpoint"))
    column_names = result.keys()
    rows = result.fetchall()
    rows = [list(row) for row in rows]
    endpoints = pl.DataFrame(rows, schema=column_names).drop([
        "export_ready", "internal_ready", "fit_all", "data_usability", "burst_assay", "key_positive_control",
        "assay_component_endpoint_desc"
    ])

endpoints = endpoints.filter(pl.col("assay_function_type") != "background control")
cell_endpoints = endpoints.filter(pl.col("acid").is_in(cell_acid))
cellfree_endpoints = endpoints.filter(pl.col("acid").is_in(cellfree_acid))

In [69]:
# join and re-order columns
col_order = [
    "assay_component_endpoint_name", "assay_component_desc",
    "cell_format", "tissue", "cell_short_name", "timepoint_hr",
    "assay_design_type", "assay_function_type", "intended_target_type", "intended_target_type_sub",
    "intended_target_family", "intended_target_family_sub", "normalized_data_type", "signal_direction",
    "cell_viability_assay", "aid", "acid", "aeid",
]

cell = cell_component.join(cell_endpoints, on="acid")
cell = cell.join(cell_assay, on="aid").select(col_order)

# separate primary endpoint from viability assays
cell_viability = cell.filter(pl.col("cell_viability_assay") == 1)
cell_other = cell.filter(pl.col("cell_viability_assay") == 0).filter(pl.col("cell_short_name") != "NA")

cellfree = cellfree_component.join(cellfree_endpoints, on="acid")
cellfree = cellfree.join(cellfree_assay, on="aid").select(col_order)

## Get assay hitcall info

In [70]:
# Get level 4 data
with engine.connect() as connection:
    result = connection.execute(text("SELECT `m4id`, `spid`, `logc_max`, `logc_min`, `nconc` FROM mc4"))
    column_names = result.keys()
    rows = result.fetchall()
    rows = [list(row) for row in rows]
    exp_info = pl.DataFrame(rows, schema=column_names)

    result = connection.execute(text("SELECT * FROM sample"))
    column_names = result.keys()
    rows = result.fetchall()
    rows = [list(row) for row in rows]
    dat = pl.DataFrame(rows, schema=column_names).select(["spid", "chid"])

exp_info = exp_info.join(dat, on="spid")
del dat
gc.collect()

0

In [71]:
# Link m5id to m4id
with engine.connect() as connection:
    result = connection.execute(text("SELECT `m5id`, `m4id` FROM mc5"))
    
    column_names = result.keys()
    rows = result.fetchall()
    rows = [list(row) for row in rows]
    m4_m5 = pl.DataFrame(rows, schema=column_names)

exp_info = exp_info.join(m4_m5, on="m4id")
del m4_m5
gc.collect()

0

In [72]:
# Get hitcall and AC50
m5id_list = exp_info.select("m5id").to_series().unique().to_list()

query = """
SELECT `m5id`, `hit_val`, `hit_param`, `aeid`
FROM mc5_param 
WHERE hit_param IN ('ac50', 'hitcall') 
AND m5id IN :m5ids
"""

with engine.connect() as connection:
    result = connection.execute(text(query), {"m5ids": tuple(m5id_list)})
    
    column_names = result.keys()
    rows = result.fetchall()
    rows = [list(row) for row in rows]
    params = pl.DataFrame(rows, schema=column_names)

params = params.pivot(
    values="hit_val",
    index=["m5id", "aeid"],
    columns="hit_param"
)

exp_info = exp_info.join(params, on="m5id")
del params
gc.collect()

0

In [73]:
# Make table for compounds
engine = create_engine("mysql+pymysql://root@localhost/prod_internal_invitrodb_v4_1")

with engine.connect() as connection:
    result = connection.execute(text("SELECT * FROM chemical"))
    column_names = result.keys()
    rows = result.fetchall()
    rows = [list(row) for row in rows]
    chemical = pl.DataFrame(rows, schema=column_names).select(["chid", "dsstox_substance_id"])

exp_info = exp_info.join(chemical, on="chid")


## Get a cytotox estimate for each chemical + relevant assay 

We only want to keep assays with matching cytotoxicity estimates. We consider a matching cytotox estimate as a viability assay run with the same chemical in either the same cell line or the same tissue. We count any cytotox hit as a hit - it's possible that viability assays that did not result in a hit tested lower concentration ranges, or a less sensitive viability assay.

In [74]:
df = cell_viability.select(["aeid", "aid", "cell_viability_assay", "cell_short_name", "tissue"])
df = df.join(exp_info, on="aeid")

df = df.with_columns(
    (pl.col("hitcall") > 0.9).cast(pl.Int64).alias("hitcall")
)

result_cell = (
    df.group_by(["chid", "cell_short_name"])
    .agg([
        pl.count().alias("ntested"),
        pl.col("hitcall").sum().alias("nhit"),
        pl.col("ac50").filter(pl.col("hitcall") == 1).median().alias("cytotox_median_ac50")
    ])
)

result_tissue = (
    df.group_by(["tissue", "chid"])
    .agg([
        pl.count().alias("ntested"),
        pl.col("hitcall").sum().alias("nhit"),
        pl.col("ac50").filter(pl.col("hitcall") == 1).median().alias("cytotox_median_ac50")
    ])
)


In [75]:
cell_chid_cytotox = cell_results.select(["chid", "cell_short_name", "tissue"]).unique()

cell_cytotox = cell_chid_cytotox.join(
    result_cell,
    on=["chid", "cell_short_name"],
    how="inner"
).with_columns(
    pl.lit("cell_short_name").alias("cytotox_source")
)

unmatched_df = cell_chid_cytotox.join(
    result_cell,
    on=["chid", "cell_short_name"],
    how="anti"
)

tissue_cytotox = unmatched_df.join(
    result_tissue,
    on=["chid", "tissue"],
    how="inner"
).with_columns(
    pl.lit("tissue").alias("cytotox_source")
)

cytotox = pl.concat([cell_cytotox, tissue_cytotox], how="vertical").rename({"ntested": "cytotox_ntested", "nhit": "cytotox_nhit"})

## Create final results info files

In [76]:
# Get OASIS IDs
oasis = pl.read_csv("../1_snakemake/inputs/annotations/seal_input/v5_oasis_03Sept2024_simple.csv")
oasis = oasis.select(["OASIS_ID", "DTXSID"]).rename({"DTXSID": "dsstox_substance_id"})

In [77]:
# cell-free assays
cellfree_results = exp_info.join(cellfree, on="aeid")

# add oasis id
cellfree_results = cellfree_results.join(oasis, on="dsstox_substance_id")

# re-order columns
cellfree_results = cellfree_results.select([
    "OASIS_ID",
    "assay_component_endpoint_name", "assay_component_desc",
    "cell_format", "tissue", "cell_short_name", "timepoint_hr",
    "assay_design_type", "assay_function_type", "intended_target_type", "intended_target_type_sub",
    "intended_target_family", "intended_target_family_sub", "normalized_data_type", "signal_direction",
    "hitcall", "ac50",
    "logc_max", "logc_min", "nconc",
    "aid", "acid", "aeid", "chid", "m4id", "m5id", "dsstox_substance_id"
])

In [78]:
# cell-based assays
cell_results = exp_info.join(cell_other, on="aeid")

# add oasis id
cell_results = cell_results.join(oasis, on="dsstox_substance_id")

# Merge assay info with cytotox estimates
cell_results = cell_results.join(cytotox, on=["chid", "cell_short_name", "tissue"])

# Re-order columns
cell_results = cell_results.select([
    "OASIS_ID",
    "assay_component_endpoint_name", "assay_component_desc",
    "cell_format", "tissue", "cell_short_name", "timepoint_hr",
    "assay_design_type", "assay_function_type", "intended_target_type", "intended_target_type_sub",
    "intended_target_family", "intended_target_family_sub", "normalized_data_type", "signal_direction",
    "hitcall", "ac50",
    "cytotox_ntested", "cytotox_nhit", "cytotox_median_ac50", "cytotox_source",
    "logc_max", "logc_min", "nconc",
    "aid", "acid", "aeid", "chid", "m4id", "m5id", "dsstox_substance_id"
])

In [94]:
# Process cytotox results for predictions
cyto_proc = pl.concat([
    result_cell.with_columns(pl.lit("cell_type").alias("cytotox_source_type")).rename({"cell_short_name": "cytotox_source"}).select(
        ["chid", "cytotox_source_type", "cytotox_source", "ntested", "nhit", "cytotox_median_ac50"]),
    result_tissue.with_columns(pl.lit("tissue").alias("cytotox_source_type")).rename({"tissue": "cytotox_source"}).select(
        ["chid", "cytotox_source_type", "cytotox_source", "ntested", "nhit", "cytotox_median_ac50"]),
], how="vertical")

# Create a label
cyto_proc = cyto_proc.with_columns(
    pl.concat_str([pl.col("cytotox_source_type"), pl.col("cytotox_source").str.replace_all(" ", "_")], separator="__").alias("assay_label")
)

# To be considered a hit, must be hit in 20% or more of tested instances
cyto_proc = cyto_proc.with_columns(
    (pl.col("nhit")/pl.col("ntested")).alias("percent_hit")
)
cyto_proc = cyto_proc.with_columns(
    pl.when(pl.col("percent_hit") < 0.2).then(None).otherwise(pl.col("cytotox_median_ac50")).alias("cytotox_median_ac50")
)

# To be considered a hit, AC50 must be lower than 100um (our highest tested dose)
cyto_proc = cyto_proc.with_columns(
    pl.when(pl.col("cytotox_median_ac50") > 100).then(None).otherwise(pl.col("cytotox_median_ac50")).alias("cytotox_median_ac50")
)

# Add OASIS IDs
cyto_proc = cyto_proc.join(chemical, on="chid", how="inner")
cyto_proc = cyto_proc.join(oasis, on="dsstox_substance_id", how="inner")

## Create binary cell-based hit file

In [340]:
# Hit: must be 2-fold lower than the cytotox ac50
cell_binary = cell_results.select([
    "OASIS_ID", "assay_component_endpoint_name", "hitcall", "ac50", "cytotox_median_ac50"
]).with_columns(
    (pl.col("hitcall") > 0.9).cast(pl.Int64).alias("hitcall"),
    (pl.col("cytotox_median_ac50")).fill_null(999999).alias("cytotox_median_ac50")
)

cell_binary = cell_binary.with_columns(
    pl.when((pl.col("cytotox_median_ac50")/2) < pl.col("ac50"))
    .then(pl.lit(0))
    .otherwise(pl.col("hitcall"))
    .alias("corr_hitcall")
)

# Sometimes there are multiple results for the same chemical-assay combo. Use majority hitcall as the hitcall. Ties go to no hit.
aggregated_df = (
    cell_binary.group_by(["OASIS_ID", "assay_component_endpoint_name"])
    .agg([
        (
            (pl.col("corr_hitcall").sum() > (pl.count() // 2))
            .cast(pl.Int64)
            .alias("corr_hitcall")
        )
    ])
)
cell_binary = cell_binary.select(["OASIS_ID", "assay_component_endpoint_name"]).unique()
cell_binary = cell_binary.join(aggregated_df, on=["OASIS_ID", "assay_component_endpoint_name"])

# pivot to one column per assay
cell_binary = cell_binary.pivot(
    values="corr_hitcall",
    index="OASIS_ID",
    columns="assay_component_endpoint_name",
)

# only keep assays with at least 5 positive and 5 negative classes
def has_min_zeros_and_ones(col):
    return (col == 0).sum() >= 5 and (col == 1).sum() >= 5

cell_binary = cell_binary.select(
    ["OASIS_ID"] + [
        col for col in cell_binary.columns if col != "OASIS_ID" and has_min_zeros_and_ones(cell_binary[col])
    ]
)

cell_assays = [i for i in cell_binary.columns if "OASIS_ID" not in i]

## Create binary cell-free hit file

In [341]:
cellfree_binary = cellfree_results.with_columns(
    (pl.col("hitcall") > 0.9).cast(pl.Int64).alias("hitcall")
).select(["OASIS_ID", "assay_component_endpoint_name", "hitcall"])

# Sometimes there are multiple results for the same chemical-assay combo. Use majority hitcall as the hitcall. Ties go to no hit.
aggregated_df = (
    cellfree_binary.group_by(["OASIS_ID", "assay_component_endpoint_name"])
    .agg([
        (
            (pl.col("hitcall").sum() > (pl.count() // 2))
            .cast(pl.Int64)
            .alias("hitcall")
        )
    ])
)
cellfree_binary = cellfree_binary.drop(["hitcall"]).unique()
cellfree_binary = cellfree_binary.join(aggregated_df, on=["OASIS_ID", "assay_component_endpoint_name"])

# pivot to one column per assay
cellfree_binary = cellfree_binary.pivot(
    values="hitcall",
    index="OASIS_ID",
    columns="assay_component_endpoint_name",
)

# only keep assays with at least 5 positive and 5 negative classes
def has_min_zeros_and_ones(col):
    return (col == 0).sum() >= 5 and (col == 1).sum() >= 5

cellfree_binary = cellfree_binary.select(
    ["OASIS_ID"] + [
        col for col in cellfree_binary.columns if col != "OASIS_ID" and has_min_zeros_and_ones(cellfree_binary[col])
    ]
)

cellfree_assays = [i for i in cellfree_binary.columns if "OASIS_ID" not in i]

In [97]:
print(cyto_proc.columns)

['chid', 'cytotox_source_type', 'cytotox_source', 'ntested', 'nhit', 'cytotox_median_ac50', 'assay_label', 'percent_hit', 'dsstox_substance_id', 'OASIS_ID']


## Create binary cytotox file

In [102]:
cytotox_binary = cyto_proc.with_columns(
    pl.when(pl.col("cytotox_median_ac50").is_not_null()).then(pl.lit(1)).otherwise(pl.lit(0)).alias("hitcall")
).select(["OASIS_ID", "assay_label", "hitcall"])

# Sometimes there are multiple results for the same chemical-assay combo. Use majority hitcall as the hitcall. Ties go to no hit.
aggregated_df = (
    cytotox_binary.group_by(["OASIS_ID", "assay_label"])
    .agg([
        (
            (pl.col("hitcall").sum() > (pl.count() // 2))
            .cast(pl.Int64)
            .alias("hitcall")
        )
    ])
)
cytotox_binary = cytotox_binary.drop(["hitcall"]).unique()
cytotox_binary = cytotox_binary.join(aggregated_df, on=["OASIS_ID", "assay_label"])

# pivot to one column per assay
cytotox_binary = cytotox_binary.pivot(
    values="hitcall",
    index="OASIS_ID",
    columns="assay_label",
)

# only keep assays with at least 5 positive and 5 negative classes
def has_min_zeros_and_ones(col):
    return (col == 0).sum() >= 5 and (col == 1).sum() >= 5

cytotox_binary = cytotox_binary.select(
    ["OASIS_ID"] + [
        col for col in cytotox_binary.columns if col != "OASIS_ID" and has_min_zeros_and_ones(cytotox_binary[col])
    ]
)

## Write out all files

In [ ]:
cell_binary.write_parquet("../1_snakemake/inputs/annotations/toxcast_cellbased_binary.parquet")
cellfree_binary.write_parquet("../1_snakemake/inputs/annotations/toxcast_cellfree_binary.parquet")
cytotox_binary.write_parquet("../1_snakemake/inputs/annotations/toxcast_cytotox_binary.parquet")

cell_results.filter(pl.col("assay_component_endpoint_name").is_in(cell_assays)).write_parquet("../1_snakemake/inputs/annotations/toxcast_cellbased_info.parquet")
cellfree_results.filter(pl.col("assay_component_endpoint_name").is_in(cellfree_assays)).write_parquet("../1_snakemake/inputs/annotations/toxcast_cellfree_info.parquet")
cyto_proc.write_parquet("../1_snakemake/inputs/annotations/toxcast_cytotox_info.parquet")